In [17]:
import gensim
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np
import re
import csv

import pandas as pd
import pprint

import string
import nltk


In [2]:
pathToDatasets = '../datasets/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
word_vectors = api.load("glove-wiki-gigaword-100")
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize



senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


/home/ubuntu/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=5)]
    
    
    return possibleReplacements

def cleanAndTokenizeText(text):
    text = text.lower()
    newString = ""
    for char in text:
        if char not in string.punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

In [11]:
def getAlternativeSentences(tweet, sentimentOfTweet):
    userInputTokens = cleanAndTokenizeText(tweet)
    print("Alternatives: ")
   
    alternativeStrings = []
    for ind,word in enumerate(userInputTokens):
        
        score = senty.polarity_scores(word)['compound']
        newUserTokens = userInputTokens[:]
        
        if(score != 0.0 ):
            replacements = listReplacements(word)       
            if(replacements == []):
                return
        
            replacements = posApprovedReplacements(replacements[:], newUserTokens[:], ind)
            print("Word being changed : {0}".format(word));
            for newWord in replacements:
                
                newUserTokens[ind] = newWord;
                newString = ' '.join(newUserTokens)
                sentimentOfNewString = senty.polarity_scores(newString)['compound']
             
            
                if(sentimentOfNewString == 0):
                    continue
                
                
                
                alternativeStrings.append(newString)
                if((sentimentOfNewString) >= (sentimentOfTweet)):
                    print("{0} : {1} ++POS++".format(newString,sentimentOfNewString))
                else:
                    print("{0} : {1} --NEG--".format(newString,sentimentOfNewString))
            newUserTokens = userInputTokens[:];
    return alternativeStrings;

In [56]:
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    tempTokens = userTokens[:]
    truePOSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = truePOSTokens[indexOfToken]
    print("Main tag : {0}".format(mainTag[1]))
#     pprint.pprint(truePOSTokens)
    
    for ind,word in enumerate(alternativeWords):
        print("Changing word:  {0} to {1}\n".format(userTokens[indexOfToken], word))
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        print("New Tag : {0}".format(newTag))
        if(str(newTag) == str(mainTag)):
            validWords.append(word)
        pprint.pprint(validWords)
    return validWords
        


def getPOSTags(tweet):
    tags = nltk.pos_tag(tweet)
    return(tags)
    

In [57]:
def runThroughTweets():
    numberOfTweets = 50;
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=1, skiprows=72 )
    tweets = tweets_df.values

    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n-NUM({2}) {0}:{1}-\n".format(tweet,mainSentiment, counter))
        newStrings = getAlternativeSentences(tweet, mainSentiment)
    
    
runThroughTweets()


-NUM(0) i really hate how people diss my bands!  trace is clearly not ugly!:0.2484-

Alternatives: 
Main tag : VB
Changing word:  hate to hatred

New Tag : VBD
[]
Changing word:  hate to racist

New Tag : VB
[]
Changing word:  hate to racism

New Tag : VB
[]
Changing word:  hate to anyone

New Tag : NN
[]
Changing word:  hate to fear

New Tag : VB
[]
Word being changed : hate
Main tag : RB
Changing word:  clearly to obviously

New Tag : RB
[]
Changing word:  clearly to certainly

New Tag : RB
[]
Changing word:  clearly to seems

New Tag : VBZ
[]
Changing word:  clearly to indeed

New Tag : RB
[]
Changing word:  clearly to seemed

New Tag : VBN
[]
Word being changed : clearly
Main tag : RB
Changing word:  ugly to nasty

New Tag : JJ
[]
Changing word:  ugly to awful

New Tag : JJ
[]
Changing word:  ugly to horrible

New Tag : JJ
[]
Changing word:  ugly to stupid

New Tag : JJ
[]
Changing word:  ugly to scary

New Tag : JJ
[]
Word being changed : ugly
